In [1]:
from argparse import ArgumentParser
from generators import PatchGenerator, PatchSequence
from keras.callbacks import Callback
from keras.models import load_model
from keras.optimizers import Adam, RMSprop
from keras.utils import multi_gpu_model
from networks import (create_initial_model,
                      create_second_model,
                      create_squeezenet3d_model,
                     )
from skimage.transform import rotate
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight
import keras.backend as K
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import os
import warnings

Using TensorFlow backend.


In [4]:
def make_augmentation_func(aug            = 0.5, 
                           aug_hflip      = True, 
                           aug_vflip      = True, 
                           aug_rotate     = True, 
                           aug_brightness = True, 
                           aug_contrast   = True, 
                           aug_zoom       = True, 
                           aug_shift      = True,
                           aug_padding    = True
                           ):
    if not aug:
        return None
    def augf(img):
        zoomed = False
        if np.random.random() > aug and aug_hflip:
            img = np.flip(img, axis = 1)
        if np.random.random() > aug and aug_vflip:
            img = np.flip(img, axis = 0)
        if np.random.random() > aug and aug_rotate:
            tmp = np.squeeze(img)
            angle = np.random.uniform(0, aug_rotate)
            tmp = rotate(tmp, angle)
            img = np.expand_dims(tmp, -1)
        if np.random.random() > aug and aug_brightness:
            up_delta = 1. - img.max()
            down_delta = img.min()
            delta = min(up_delta, down_delta)
            img = img + np.random.uniform(-delta, delta)    
        if np.random.random() > aug and aug_contrast:
            tmp = np.reshape(img, img.shape[0], img.shape[1]*img.shape[2])
            tmp = equalize_adapthist(tmp, kernel_size=(5,5,5,1))
            img = np.reshape(tmp, 70, 70, 40, 1)

        if np.random.random() > aug and aug_zoom:
            img = zoom(img, (*np.random.choice([0.9, 1.1], 2), 1,1))
            zoomed = True

        crop_from = [15, 15]
        if np.random.random() > aug and aug_shift:
            if zoomed:
                crop_from = np.random.randint(0, 23, 2)
            else:
                crop_from = np.random.randint(0, 30, 2)

        # Padding is only over cropped image
        img = img[crop_from[0]:crop_from[0] + 40, crop_from[1]:crop_from[1] + 40, :]
        
        if np.random.random() > aug and aug_padding:
            try:
                direction = np.random.randint(0,2)
                start = np.random.choice([0,20])
                size = np.random.randint(0,20)
                if direction == 0:
                    img[start:size+start, :, :] = 0
                elif direction == 1:
                    img[:, start:size+start, :] = 0
                else:
                    img[:,:, start:size+start] = 0
            except Exception:
                print("padding augmentation failed")
            
        return img

    return augf

# Single stage training

In [4]:
NETWORKS = {
    'initial': create_initial_model,
    'second': create_second_model,
    'squeezenet3d': create_squeezenet3d_model,
}

LOSS_FUNCTION = 'categorical_crossentropy'

OPTIMIZERS = {
    'adam': Adam,
    'rmsprop': RMSprop,
}

DATADIR = '/projects/0/ismi2018/FINALPROJECTS/BREAST_3D_ULTRASOUND/shareWithStudents'

class MultiGPUCheckpoint(Callback):

    def __init__(self, filename, verbose=0):
        super().__init__()
        self.filename = filename
        self.verbose = verbose
        self.val_accs = []

    def on_epoch_end(self, epoch, logs=None):
        if not self.val_accs:
            self.model.layers[-2].save(self.filename)
        elif logs['val_acc'] > max(self.val_accs):
            if self.verbose > 0:
                print('Saving to {}'.format(self.filename))
            self.model.layers[-2].save(self.filename)
        self.val_accs.append(logs['val_acc'])


class Accuracies(Callback):

    def __init__(self, valid_seq):
        super().__init__()
        self.valid_seq = valid_seq
        self.label_accuracies = []

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict_generator(self.valid_seq,
                                              workers=4,
                                              use_multiprocessing=True)
        y_pred = np.argmax(y_pred, axis=1)

        y_true = self.valid_seq.get_all_labels()
        y_true[y_true == 2] = 0
        y_true[y_true == 20] = 1
        y_true[y_true == 21] = 2

        cm = confusion_matrix(y_true, y_pred)
        ps = cm.diagonal() / cm.sum(axis=1)
        self.label_accuracies.append(ps)


def create_model(network, optimizer, drop_rate, multi_gpu):
    orig_model = NETWORKS[network](drop_rate=drop_rate)
    if multi_gpu:
        parallel_model = multi_gpu_model(orig_model)
        parallel_model.compile(optimizer=optimizer, loss=LOSS_FUNCTION,
                               metrics=['accuracy'])
    else:
        orig_model.compile(optimizer=optimizer, loss=LOSS_FUNCTION,
                           metrics=['accuracy'])
        parallel_model = None
    return orig_model, parallel_model


def create_optimizer(name, lr, decay):
    return OPTIMIZERS[name](lr=lr, decay=decay)


def make_generators(csv, train_patients, validation_patients, batch_size,
                    augf):
    train_csv = csv.loc[csv['patientID'].isin(train_patients), :]
    valid_csv = csv.loc[csv['patientID'].isin(validation_patients), :]

    train_gen = PatchGenerator(
        input_dir=DATADIR,
        dataframe=train_csv,
        batch_size=batch_size,
        augmentation_fn=augf
    )

    valid_seq = PatchSequence(
        input_dir=DATADIR,
        dataframe=valid_csv,
        batch_size=batch_size
    )

    return train_gen, valid_seq


def train_model(augmentations, filename):
    csv = pd.read_csv(os.path.join(DATADIR, 'trainingSet.csv'), dtype=str)


    # Create patient K-folder
    unique_patients = csv.patientID.unique()
    kf = KFold(5, shuffle=True, random_state=42)
    folds = kf.split(unique_patients)

    # Make augmentation function
    augf = make_augmentation_func(*augmentations)

    class_weight = False
    if class_weight:
        cw = compute_class_weight('balanced',
                                  np.unique(csv['histology'].values),
                                  csv['histology'].values)
        cwdict = dict(enumerate(cw))
    else:
        cwdict = None

    accuracies = []
    for i, (train_idxs, val_idxs) in enumerate(folds, start=1):
        K.clear_session()
        print('Fold {}'.format(i))

        train_patients = unique_patients[train_idxs]
        print(train_patients)
        val_patients = unique_patients[val_idxs]

        train_gen, valid_seq = make_generators(csv,
                                               train_patients,
                                               val_patients,
                                               6,
                                               augf)
        print(train_gen)
        optimizer = create_optimizer('adam', 1e-5, 1e-6)
                
        orig_net, parallel_net = create_model('squeezenet3d', optimizer,
                                              0.5,
                                              True)

        save_filename = '{}_fold_{}.h5'.format(filename, i)
        
        if True:
            cp = MultiGPUCheckpoint(save_filename, verbose=1)
        else:
            cp = ModelCheckpoint(save_filename, save_best_only=True, verbose=1,
                                 monitor='val_acc')
        ps = Accuracies(valid_seq)

        train_model = parallel_net or orig_net
        results = train_model.fit_generator(train_gen,
                                            steps_per_epoch=len(train_gen),
                                            validation_data=valid_seq,
                                            epochs=50,
                                            use_multiprocessing=True,
                                            workers=4,
                                            class_weight=cwdict,
                                            callbacks=[cp, ps],
                                            verbose=1)

        h = results.history
        plt.figure()
        plt.plot(h['loss'])
        plt.plot(h['acc'])
        plt.plot(h['val_loss'])
        plt.plot(h['val_acc'])
        plt.legend(['loss', 'acc', 'val_loss', 'val_acc'])
        plt.savefig('{}.traininglog.png'.format(save_filename))

        y_true = valid_seq.get_all_labels()
        y_true[y_true == 2] = 0
        y_true[y_true == 20] = 1
        y_true[y_true == 21] = 2
        best_net = load_model(save_filename)
        y_pred = best_net.predict_generator(valid_seq,
                                            workers=4,
                                            use_multiprocessing=True)
        y_pred = np.argmax(y_pred, axis=1)
        cm = confusion_matrix(y_true, y_pred)

        plt.figure()
        sns.heatmap(cm, annot=True)
        plt.savefig('{}.confusionmatrix.png'.format(save_filename))

        precs = np.array(ps.label_accuracies)

        plt.figure()
        for i in range(precs.shape[1]):
            plt.plot(precs[:, i])
        plt.legend(['0', '1', '2'])
        plt.savefig('{}.accuracies.png'.format(save_filename))

        accuracies.append(max(h['val_acc']))

    with open('{}_score.txt'.format(filename), 'w') as f:
        print('Mean accuracy: {:.4f}'.format(np.mean(accuracies)), file=f)

In [ ]:
# aug, aug_hflip, aug_vflip, aug_rotate, aug_brightness, aug_contrast, aug_zoom, aug_shift

# No augmentation
augs = (False,)
train_model(augs, "no_augs")

Fold 1
['02000801' '02002243' '02002287' '02002117' '02001283' '02004004'
 '02001241' '02000699' '02001238' '02001798' '02000740' '02000757'
 '02001839' '02002118' '02004362' '02000735' '02001746' '02004189'
 '02000731' '02001951' '02002166' '02000903' '02004418' '02002527'
 '02004437' '02000751' '02002560' '02001909' '02009241' '02001305'
 '02000500' '02001293' '02001877' '02001881' '02001233' '02001813'
 '02001743' '02002076' '02004171' '02001830' '02004033' '02008917'
 '02006939' '02004427' '02001312' '02002238' '02001560' '02001261'
 '02001818' '02000714' '02000739' '02004475' '02000736' '02000729'
 '02001791' '02000678' '02009188' '02002069' '02000688' '02001386'
 '02001774' '02002545' '02000704' '02002800' '02004214' '02000727'
 '02000081' '02001833' '02001254' '02000703' '02002171' '02001944'
 '02000765' '02001285' '02001167' '02009163' '02001313' '02000289'
 '02002020' '02004242' '02000732' '02000838' '02002563' '02001276'
 '02004028' '02002540' '02000764' '02002132' '02000758'

/home/ruc0027/.local/lib/python3.5/site-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/50
Failed reading idx 160...
Failed reading idx 136...
Failed reading idx 279...
Failed reading idx 150...
Failed reading idx 297...
Failed reading idx 9...
Failed reading idx 210...
Failed reading idx 84...
Failed reading idx 217...
Failed reading idx 147...
Failed reading idx 219...
Failed reading idx 218...
Failed reading idx 218...
Failed reading idx 6...
Failed reading idx 144...
Failed reading idx 234...
Failed reading idx 203...
Failed reading idx 288...


  File "/nfs/home1/ruc0027/ISMI-Project/generators.py", line 77, in next
    batch_x = np.stack(images).astype(K.floatx())
Traceback (most recent call last):
  File "/home/ruc0027/.local/lib/python3.5/site-packages/keras/utils/data_utils.py", line 677, in _data_generator_task
    generator_output = next(self._generator)
  File "/nfs/home1/ruc0027/ISMI-Project/generators.py", line 31, in __next__
    return self.next()
  File "/hpc/sw/python-3.5.2/lib/python3.5/site-packages/numpy/core/shape_base.py", line 343, in stack
    raise ValueError('need at least one array to stack')
ValueError: need at least one array to stack


Failed reading idx 94...
Failed reading idx 10...
Failed reading idx 244...
Failed reading idx 228...
Failed reading idx 147...
Failed reading idx 266...
Failed reading idx 216...
Failed reading idx 78...
Failed reading idx 210...
Failed reading idx 26...
Failed reading idx 92...
Failed reading idx 123...


Traceback (most recent call last):
  File "/home/ruc0027/.local/lib/python3.5/site-packages/keras/utils/data_utils.py", line 677, in _data_generator_task
    generator_output = next(self._generator)
  File "/nfs/home1/ruc0027/ISMI-Project/generators.py", line 31, in __next__
    return self.next()
  File "/nfs/home1/ruc0027/ISMI-Project/generators.py", line 77, in next
    batch_x = np.stack(images).astype(K.floatx())
  File "/hpc/sw/python-3.5.2/lib/python3.5/site-packages/numpy/core/shape_base.py", line 343, in stack
    raise ValueError('need at least one array to stack')
ValueError: need at least one array to stack


Failed reading idx 39...
Failed reading idx 26...


## With padding

In [ ]:
# # Only padding
# augs = (0.5, False, False, False, False, False, False, False, True)
# train_model(augs, "padding_only")

In [ ]:
# # Only flipping and rotation
# augs = (0.5, True, True, True, False, False, False, False, True)
# train_model(augs, "padding_flip_rot")

In [ ]:
# # Only brightness
# augs = (0.5, False, False, False, True, False, False, False, True)
# train_model(augs, "padding_brightness")

In [ ]:
# # Only contrast
# augs = (0.5, False, False, False, False, True, False, False, True)
# train_model(augs, "padding_contrast")

In [ ]:
# # Only zoom
# augs = (0.5, False, False, False, False, False, True, False, True)
# train_model(augs, "padding_zoom")

In [ ]:
# # All augmentations
# augs = (0.5, True, True, True, True, True, True, False, True)
# train_model(augs, "padding_all_augs")

## With shifting

In [ ]:
# Only padding
augs = (0.5, False, False, False, False, False, False, True, False)
train_model(augs, "shift_only")

In [ ]:
# Only flipping and rotation
augs = (0.5, True, True, True, False, False, False, True, False)
train_model(augs, "shift_flip_rot")

In [ ]:
# Only brightness
augs = (0.5, False, False, False, True, False, False, True, False)
train_model(augs, "shift_brightness")

In [ ]:
# Only contrast
augs = (0.5, False, False, False, False, True, False, True, False)
train_model(augs, "shift_contrast")

In [ ]:
# Only zoom
augs = (0.5, False, False, False, False, False, True, True, False)
train_model(augs, "shift_zoom")

In [ ]:
# All augmentations
augs = (0.5, True, True, True, True, True, True, True, False)
train_model(augs, "shift_all_augs")

# two stage

In [5]:
from argparse import ArgumentParser
from generators_binary import PatchGenerator, PatchSequence
from keras.callbacks import Callback
from keras.models import load_model
from keras.optimizers import Adam, RMSprop
from keras.utils import multi_gpu_model
from networks_binary import (create_initial_model,
                      create_second_model,
                      create_squeezenet3d_model,
                      create_squeezenet3d_model2
                     )
from skimage.transform import rotate
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight

# machine learning / deep learning
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.Session(config=config)

import keras.backend as K
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys, os, warnings

DATADIR = '/projects/0/ismi2018/FINALPROJECTS/BREAST_3D_ULTRASOUND/shareWithStudents'

NETWORKS = {
    'initial': create_initial_model,
    'second': create_second_model,
    'squeezenet3d': create_squeezenet3d_model,
    'squeezenet3d2': create_squeezenet3d_model2
}

LOSS_FUNCTION = 'categorical_crossentropy'

OPTIMIZERS = {
    'adam': Adam,
    'rmsprop': RMSprop,
}


class MultiGPUCheckpoint(Callback):

    def __init__(self, filename, verbose=0):
        super().__init__()
        self.filename = filename
        self.verbose = verbose
        self.val_loss = []

    def on_epoch_end(self, epoch, logs=None):
        if not self.val_loss:
            self.model.layers[-2].save(self.filename)
        elif logs['val_loss'] < min(self.val_loss):
            if self.verbose > 0:
                print('Saving to {}'.format(self.filename))
            self.model.layers[-2].save(self.filename)
        self.val_loss.append(logs['val_loss'])


class Accuracies(Callback):

    def __init__(self, valid_seq, step = 0):
        super().__init__()
        self.valid_seq = valid_seq
        self.label_accuracies = []
        self.step = step

    def on_epoch_end(self, epoch, logs=None):
        
        y_pred = self.model.predict_generator(self.valid_seq,
                                              workers=4,
                                              use_multiprocessing=True)
        y_pred = np.argmax(y_pred, axis=1)
        y_true = self.valid_seq.get_all_labels()
        
        if self.step > 0:
            y_true = y_true[y_true != 21]
            y_true[y_true == 2] = 1
            y_true[y_true == 20] = 0
        else:
            y_true[y_true == 2] = 1
            y_true[y_true == 20] = 1
            y_true[y_true == 21] = 0
        
        
        cm = confusion_matrix(y_true, y_pred)
        ps = cm.diagonal() / cm.sum(axis=1)
        self.label_accuracies.append(ps)


def create_model(network, optimizer, drop_rate, multi_gpu):
    orig_model = NETWORKS[network](drop_rate=drop_rate)
    if multi_gpu:
        parallel_model = multi_gpu_model(orig_model)
        parallel_model.compile(optimizer=optimizer, loss=LOSS_FUNCTION,
                               metrics=['accuracy'])
    else:
        orig_model.compile(optimizer=optimizer, loss=LOSS_FUNCTION,
                           metrics=['accuracy'])
        parallel_model = None
    return orig_model, parallel_model


def create_optimizer(name, lr, decay):
    return OPTIMIZERS[name](lr=lr, decay=decay)


def make_generators(csv, train_patients, validation_patients, batch_size, step,
                    augf):
    train_csv = csv.loc[csv['patientID'].isin(train_patients), :]
    valid_csv = csv.loc[csv['patientID'].isin(validation_patients), :]

    train_gen = PatchGenerator(
        input_dir=DATADIR,
        dataframe=train_csv,
        batch_size=batch_size,
        step=step,
        augmentation_fn=augf
    )

    valid_seq = PatchSequence(
        input_dir=DATADIR,
        dataframe=valid_csv,
        batch_size=batch_size,
        step=step
    )

    return train_gen, valid_seq


def train_model_1st(augmentations, filename):
    csv = pd.read_csv(os.path.join(DATADIR, 'trainingSet.csv'), dtype=str)

    # Create patient K-folder
    unique_patients = csv.patientID.unique()
    kf = KFold(5, shuffle=True, random_state=42)
    folds = kf.split(unique_patients)

    # Make augmentation function
    augf = make_augmentation_func(*augmentations)
    
    # -----------------------------------------------------------------------
        
    accuracies_first = []
    # first step training
    for i, (train_idxs, val_idxs) in enumerate(folds, start=1):
        K.clear_session()
        print('Fold {}'.format(i))

        train_patients = unique_patients[train_idxs]
        val_patients = unique_patients[val_idxs]

        train_gen, valid_seq = make_generators(csv,
                                               train_patients,
                                               val_patients,
                                               6,
                                               0,
                                               augf)
        
        optimizer = create_optimizer('adam', 1e-5, 1e-6)
        orig_net, parallel_net = create_model('squeezenet3d', optimizer,
                                              0.5,
                                              True)

        save_filename = './results_1st/{}_fold_{}.h5'.format(filename, i)
        
        if True:
            cp = MultiGPUCheckpoint(save_filename, verbose=1)
        else:
            cp = ModelCheckpoint(save_filename, save_best_only=True, verbose=1,
                                 monitor='val_acc')
        ps = Accuracies(valid_seq, 0)

        train_model = parallel_net or orig_net
                
        results = train_model.fit_generator(train_gen,
                                            steps_per_epoch=len(train_gen),
                                            validation_data=valid_seq,
                                            epochs=100,
                                            workers=4,
                                            use_multiprocessing=True,
                                            callbacks=[cp, ps],
                                            verbose=1)

        h = results.history
        plt.figure()
        plt.plot(h['loss'])
        plt.plot(h['acc'])
        plt.plot(h['val_loss'])
        plt.plot(h['val_acc'])
        plt.legend(['loss', 'acc', 'val_loss', 'val_acc'])
        plt.savefig('{}.traininglog.png'.format(save_filename))

        y_true = valid_seq.get_all_labels()
        y_true[y_true == 2] = 1
        y_true[y_true == 20] = 1
        y_true[y_true == 21] = 0
        best_net = load_model(save_filename)
        y_pred = best_net.predict_generator(valid_seq,
                                            workers=4,
                                            use_multiprocessing=True)
        y_pred = np.argmax(y_pred, axis=1)
        cm = confusion_matrix(y_true, y_pred)

        plt.figure()
        sns.heatmap(cm, annot=True)
        plt.savefig('{}.confusionmatrix.png'.format(save_filename))

        precs = np.array(ps.label_accuracies)

        plt.figure()
        for i in range(precs.shape[1]):
            plt.plot(precs[:, i])
        plt.legend(['0', '1'])
        plt.savefig('{}.accuracies.png'.format(save_filename))

        accuracies_first.append(max(h['val_acc']))

    with open('{}_score.txt'.format(filename), 'w') as f:
        print('Mean accuracy 1st step: {:.4f}\n'.format(np.mean(accuracies_first)), file=f)
        

def train_model_2nd(augmentations, filename):
    csv = pd.read_csv(os.path.join(DATADIR, 'trainingSet.csv'), dtype=str)

    # Create patient K-folder
    unique_patients = csv.patientID.unique()
    kf = KFold(5, shuffle=True, random_state=42)
    folds = kf.split(unique_patients)

    # Make augmentation function
    augf = make_augmentation_func(*augmentations)
                
    # -------------------------------------------------------------------------
    accuracies_second = []
    # second step training
    for i, (train_idxs, val_idxs) in enumerate(folds, start=1):
        K.clear_session()
        print('Fold {}'.format(i))

        train_patients = unique_patients[train_idxs]
        val_patients = unique_patients[val_idxs]

        train_gen, valid_seq = make_generators(csv,
                                               train_patients,
                                               val_patients,
                                               6,
                                               1,
                                               augf)
        
        optimizer = create_optimizer('adam', 1e-5, 1e-6)
        orig_net, parallel_net = create_model('squeezenet3d', optimizer,
                                              0.5,
                                              True)

        save_filename = './results_2nd/{}_fold_{}.h5'.format(filename, i)
        if True:
            cp = MultiGPUCheckpoint(save_filename, verbose=1)
        else:
            cp = ModelCheckpoint(save_filename, save_best_only=True, verbose=1,
                                 monitor='val_acc')
        ps = Accuracies(valid_seq, 1)

        train_model = parallel_net or orig_net
                
        results = train_model.fit_generator(train_gen,
                                            steps_per_epoch=len(train_gen),
                                            validation_data=valid_seq,
                                            epochs=100,
                                            workers=4,
                                            use_multiprocessing=True,
                                            callbacks=[cp, ps],
                                            verbose=1)

        h = results.history
        plt.figure()
        plt.plot(h['loss'])
        plt.plot(h['acc'])
        plt.plot(h['val_loss'])
        plt.plot(h['val_acc'])
        plt.legend(['loss', 'acc', 'val_loss', 'val_acc'])
        plt.savefig('{}.traininglog.png'.format(save_filename))

        y_true = valid_seq.get_all_labels()
        y_true = y_true[y_true != 21]
        y_true[y_true == 2] = 1
        y_true[y_true == 20] = 0
        best_net = load_model(save_filename)
        y_pred = best_net.predict_generator(valid_seq,
                                            workers=4,
                                            use_multiprocessing=True)
        y_pred = np.argmax(y_pred, axis=1)
        cm = confusion_matrix(y_true, y_pred)

        plt.figure()
        sns.heatmap(cm, annot=True)
        plt.savefig('{}.confusionmatrix.png'.format(save_filename))

        precs = np.array(ps.label_accuracies)

        plt.figure()
        for i in range(precs.shape[1]):
            plt.plot(precs[:, i])
        plt.legend(['0', '1'])
        plt.savefig('{}.accuracies.png'.format(save_filename))

        accuracies_second.append(max(h['val_acc']))
    
    with open('{}_score.txt'.format(filename), 'w') as f:
        print('Mean accuracy 1st step: {:.4f}\n'.format(np.mean(accuracies_second)), file=f)

InternalError: Failed to create session.

In [ ]:
# aug, aug_hflip, aug_vflip, aug_rotate, aug_brightness, aug_contrast, aug_zoom, aug_shift

# No augmentation
augs = (False,)
train_model_1st(augs, "no_augs_st1")
train_model_2nd(augs, "no_augs_st2")

## padding

In [ ]:
# # Only padding
# augs = (0.5, False, False, False, False, False, False, False, True)
# train_model_1st(augs, "padding_only_st1")
# train_model_2nd(augs, "padding_only_st2")

In [ ]:
# # Only flipping and rotation
# augs = (0.5, True, True, True, False, False, False, False, True)
# train_model_1st(augs, "padding_flip_rot_st1")
# train_model_2nd(augs, "padding_flip_rot_st2")

In [ ]:
# # Only brightness
# augs = (0.5, False, False, False, True, False, False, False, True)
# train_model_1st(augs, "padding_brightness_st1")
# train_model_2nd(augs, "padding_brightness_st2")

In [ ]:
# # Only contrast
# augs = (0.5, False, False, False, False, True, False, False, True)
# train_model_1st(augs, "padding_contrast_st1")
# train_model_2nd(augs, "padding_contrast_st2")

In [ ]:
# # Only zoom
# augs = (0.5, False, False, False, False, False, True, False, True)
# train_model_1st(augs, "padding_zoom_st1")
# train_model_2nd(augs, "padding_zoom_st2")

In [ ]:
# # All augmentations
# augs = (0.5, True, True, True, True, True, True, False, True)
# train_model_1st(augs, "padding_all_augs_st1")
# train_model_2nd(augs, "padding_all_augs_st2")

## shifting

In [ ]:
# Only padding
augs = (0.5, False, False, False, False, False, False, True, False)
train_model_1st(augs, "shift_only_st1")
train_model_2nd(augs, "shift_only_st2")

In [ ]:
# Only flipping and rotation
augs = (0.5, True, True, True, False, False, False, True, False)
train_model_1st(augs, "shift_flip_rot_st1")
train_model_2nd(augs, "shift_flip_rot_st2")

In [ ]:
# Only brightness
augs = (0.5, False, False, False, True, False, False, True, False)
train_model_1st(augs, "shift_brightness_st1")
train_model_2nd(augs, "shift_brightness_st2")

In [ ]:
# Only contrast
augs = (0.5, False, False, False, False, True, False, True, False)
train_model_1st(augs, "shift_contrast_st1")
train_model_2nd(augs, "shift_contrast_st2")

In [ ]:
# Only zoom
augs = (0.5, False, False, False, False, False, True, True, False)
train_model_1st(augs, "shift_zoom_st1")
train_model_2nd(augs, "shift_zoom_st2")

In [ ]:
# All augmentations
augs = (0.5, True, True, True, True, True, True, True, False)
train_model_1st(augs, "shift_all_augs_st1")
train_model_2nd(augs, "shift_all_augs_st2")